Attention Is All You Need (Transformer) 是当今深度学习初学者必读的一篇论文。 这篇工作当时主要是用于解决机器翻译问题，这里先补充/复习下背景知识，再清晰地解读一下这篇论文

## 知识准备

机器翻译，就是将某种语言的一段文字翻译成另一段文字。

由于翻译没有唯一的正确答案，用准确率来衡量一个机器翻译算法并不合适。因此，机器翻译的数据集通常会为每一条输入准备若干个参考输出。统计算法输出和参考输出之间的重复程度，就能评价算法输出的好坏了。这种评价指标叫做BLEU Score。这一指标越高越好。

![BLEU Score例子](./images/Transformer/p3.png)

编码器和解码器之间只通过一个隐状态来传递信息。在处理较长的文章时，这种架构的表现不够理想。为此，有人提出了基于注意力的架构。这种架构依然使用了编码器和解码器，只不过解码器的输入是编码器的状态的加权和，而不再是一个简单的中间状态。每一个输出对每一个输入的权重叫做注意力，注意力的大小取决于输出和输入的相关关系。这种架构优化了编码器和解码器之间的信息交流方式，在处理长文章时更加有效。

尽管注意力模型的表现已经足够优秀，但所有基于RNN的模型都面临着同样一个问题：RNN本轮的输入状态取决于上一轮的输出状态，这使RNN的计算必须串行执行。因此，RNN的训练通常比较缓慢。在这一背景下，抛弃RNN，只使用注意力机制的Transformer横空出世了。

## 摘要与引言

摘要传递的信息非常简练：
- 当前最好的架构是基于注意力的"encoder-decoder"架构。这些架构都使用了CNN或RNN。这篇文章提出的Transformer架构仅使用了注意力机制，而无需使用CNN和RNN。
- 机器翻译的实验表明，这种架构不仅精度高，而且训练时间大幅缩短。摘要并没有解释Transformer的设计动机。让我们在引言中一探究竟。

摘要并没有解释Transformer的设计动机。在引言中探索。

引言的第一段回顾了RNN架构。以LSTM和GRU为代表的RNN在多项序列任务中取得顶尖的成果。许多研究仍在拓宽循环语言模型和"encoder-decoder"架构的能力边界。

第二段就开始讲RNN的不足了。RNN要维护一个隐状态，该隐状态取决于上一时刻的隐状态。这种内在的串行计算特质阻碍了训练时的并行计算（特别是训练序列较长时，每一个句子占用的存储更多，batch size变小，并行度降低）。有许多研究都在尝试解决这一问题，但是，串行计算的本质是无法改变的。

上一段暗示了Transformer的第一个设计动机：提升训练的并行度。第三段讲了Transformer的另一个设计动机：注意力机制。注意力机制是当时最顶尖的模型中不可或缺的组件。这一机制可以让每对输入输出关联起来，而不用像早期使用一个隐状态传递信息的"encoder-decoder"模型一样，受到序列距离的限制。然而，几乎所有的注意力机制都用在RNN上的。

既然注意力机制能够无视序列的先后顺序，捕捉序列间的关系，为什么不只用这种机制来构造一个适用于并行计算的模型呢？因此，在这篇文章中，作者提出了Transformer架构。这一架构规避了RNN的使用，完全使用注意力机制来捕捉输入输出序列之间的依赖关系。这种架构不仅训练得更快了，表现还更强了。

通过阅读摘要和引言，我们基本理解了Transformer架构的设计动机。作者想克服RNN不能并行的缺点，又想充分利用没有串行限制的注意力机制，于是就提出了一个只有注意力机制的模型。模型训练出来了，结果出乎预料地好，不仅训练速度大幅加快，模型的表现也超过了当时所有其他模型。

## 注意力机制

### 例子

先关注架构里的核心机制，再来看Transformer的整体架构

其实，“注意力”这个名字取得非常不易于理解。这个机制应该叫做“全局信息查询”。做一次“注意力”计算，其实就跟去数据库了做了一次查询一样。假设，我们现在有这样一个以人名为key（键），以年龄为value（值）的数据库：

```
{
    张三: 18,
    张三: 20,
    李四: 22,
    张伟: 19
}
```

现在，我们有一个query（查询），问所有叫“张三”的人的年龄平均值是多少。让我们写程序的话，我们会把字符串“张三”和所有key做比较，找出所有“张三”的value，把这些年龄值相加，取一个平均数。这个平均数是(18+20)/2=19。

但是，很多时候，我们的查询并不是那么明确。比如，我们可能想查询一下所有姓张的人的年龄平均值。这次，我们不是去比较key == 张三,而是比较key[0] == 张。这个平均数应该是(18+20+19)/3=19。

或许，我们的查询会更模糊一点，模糊到无法用简单的判断语句来完成。因此，最通用的方法是，把query和key各建模成一个向量。之后，对query和key之间算一个相似度（比如向量内积），以这个相似度为权重，算value的加权和。这样，不管多么抽象的查询，我们都可以把query, key建模成向量，用向量相似度代替查询的判断语句，用加权和代替直接取值再求平均值。“注意力”，其实指的就是这里的权重。

把这种新方法套入刚刚那个例子里。我们先把所有key建模成向量，可能可以得到这样的一个新数据库：

```
{
    [1, 2, 0]: 18, # 张三
    [1, 2, 0]: 20, # 张三
    [0, 0, 2]: 22, # 李四
    [1, 4, 0]: 19 # 张伟
}
```

假设key[0]==1表示姓张。我们的查询“所有姓张的人的年龄平均值”就可以表示成向量[1, 0, 0]。用这个query和所有key算出的权重是：
```
dot([1, 0, 0], [1, 2, 0]) = 1
dot([1, 0, 0], [1, 2, 0]) = 1
dot([1, 0, 0], [0, 0, 2]) = 0
dot([1, 0, 0], [1, 4, 0]) = 1
```

之后，我们该用这些权重算平均值了。注意，算平均值时，权重的和应该是1。因此，我们可以用softmax把这些权重归一化一下，再算value的加权和。

```
softmax([1, 1, 0, 1]) = [1/3, 1/3, 0, 1/3]
dot([1/3, 1/3, 0, 1/3], [18, 20, 22, 19]) = 19
```
这样，我们就用向量运算代替了判断语句，完成了数据库的全局信息查询。那三个1/3，就是query对每个key的注意力。

### 缩放点乘注意力

我们刚刚完成的计算差不多就是Transformer里的注意力，这种计算在论文里叫做放缩点乘注意力（Scaled Dot-Product Attention）。它的公式是：

$$
\mathrm{Attention}(Q, K, V)
=
\mathrm{softmax}\!\left(
\frac{QK^{T}}{\sqrt{d_k}}
\right)
V
$$

我们先来看看Q,K,V在刚刚那个例子里究竟是什么。比较好理解，其实就是key向量的数组，也就是

```
K = [[1, 2, 0], [1, 2, 0], [0, 0, 2], [1, 4, 0]]
```

同样，V就是value向量的数组。而在我们刚刚那个例子里，value都是实数。实数其实也就是可以看成长度为1的向量。因此，那个例子的V应该是

```
V = [[18], [20], [22], [19]]
```

在刚刚那个例子里，我们只做了一次查询。因此，准确来说，我们的操作应该写成:

$$
\mathrm{MyAttention}(q, K, V) = \mathrm{softmax}(qK^{T})V
$$

其中，query $q$ 就是 $[1, 0, 0]^{T}$。

实际上，我们可以一次做多组 query。把所有 $q$ 打包成矩阵 $Q$，就得到了公式：

$$
\mathrm{Attention}(Q, K, V)
=
\mathrm{softmax}\!\left(
\frac{QK^{T}}{\sqrt{d_k}}
\right)V
$$

$d_k$ 就是 query 和 key 向量的长度。由于 query 和 key 要做点乘，这两种向量的长度必须一致。value 向量的长度则是可以不一致，论文里把 value 向量的长度叫做 $d_v$。在我们这个例子里，$d_k = 3$，$d_v = 1$。

为什么要用一个和 $d_k$ 成比例的项来缩放 $QK^{T}$ 呢？这是因为，softmax 在绝对值较大的区域梯度较小，梯度下降速度比较慢。因此，我们希望送入 softmax 的点乘数值尽可能小。而一般在 $d_k$ 较大时（也就是向量较长时），点乘的数值会比较大。除以一个和 $d_k$ 相关的量能够防止点乘的值过大。

刚才也提到，$QK^{T}$ 其实是在算 query 和 key 的相似度。而算相似度并不只有点乘这一种方式。另一种常用的注意力函数叫做**加性注意力**，它用一个单层神经网络来计算两个向量的相似度。相比之下，点乘注意力算起来快一些。出于性能上的考量，论文使用了点乘注意力。


### 自注意力

大致明白了注意力机制其实就是“全局信息查询”，并掌握了注意力的公式后，我们来以 Transformer 的自注意力为例，进一步理解注意力的意义。

自注意力模块的目的是为每一个输入 token 生成一个向量表示，该表示不仅能反映 token 本身的性质，还能反映 token 在句子中所特有的性质。比如翻译“简访问非洲”这句话时，第三个字“问”在中文里有很多个意思，比如询问、慰问等。我们想为它生成一个表示，知道它在句子中的具体意思。而在例句中，“问”字和前面组成了“访问”，所以它应该取“询问”这个意思，而不是“慰问”。“询问”就是“问”字在这句话里的表示。

让我们看看自注意力模块具体是怎么生成这种表示的。自注意力模块的输入是 3 个矩阵 $Q$、$K$、$V$。准确来说，这些矩阵是向量的数组，也就是每一个 token 的 query、key、value 向量构成的数组。自注意力模块会为每一个 token 输出一个向量表示 $A$。$A^{<t>}$ 是第 $t$ 个 token 在这句话里的向量表示。（先不管token的query, key, value究竟是什么算出来的，后文会对此做解释。）

![自注意力](./images/Transformer/p4.png)

让我们还是以刚刚那个句子“简访问非洲”为例，看一下自注意力是怎么计算的。  现在，我们想计算 $A^{<3>}$。  $A^{<3>}$ 表示的是“问”字在句子里的确切含义。

为了获取 $A^{<3>}$，我们可以问这样一个可以用数学表达的问题：  “和‘问’字组词的字的嵌入是什么？”。  这个问题就是第三个 token 的 query 向量 $q^{<3>}$。

和“问”字组词的字，很可能是一个动词。  恰好，每一个 token 的 key $k^{<t>}$ 就表示这个 token 的词性；  每一个 token 的 value $v^{<t>}$，就是这个 token 的嵌入。

这样，我们就可以根据每个字的词性（key），尽量去找动词（和 query 比较相似的 key），求出权重（query 和 key 做点乘再做 softmax），对所有 value 求一个加权平均，就差不多能回答问题 $q^{<3>}$ 了。

经计算，$q^{<3>}$、$k^{<2>}$ 可能会比较相关，即这两个向量的内积比较大。因此，最终算出来的 $A^{<3>}$ 应该等于 $v^{<2>}$，即问题“哪个字和‘问’字组成了‘访问’？”的答案是第一个字“访”。

![自注意力2](./images/Transformer/p5.png)

这是 $A^{<3>}$ 的计算过程。准确来说，
$A^{<3>} = A(q^{<3>}, K, V)$。
类似地，$A^{<1>}$ 到 $A^{<5>}$ 都是用这个公式计算。

把所有 $A$ 的计算合起来，把 $q$ 合起来，得到的公式就是注意力的公式：

$$
\mathrm{Attention}(Q, K, V)
=
\mathrm{softmax}\!\left(
\frac{QK^{T}}{\sqrt{d_k}}
\right)V
$$

从上一节中，我们知道了注意力其实是**全局信息查询**。而在这一节中，我们知道了注意力的一种应用：通过让一句话中的每个单词去向其他单词查询信息，我们能为每一个单词生成一个**更有意义的向量表示**。

可是，我们还留了一个问题没有解决：每个单词的 query、key、value 是怎么得到的？这就要看 Transformer 里的另一种机制——**多头注意力**。


### 多头注意力

在自注意力中，每一个单词的 query、key、value 应该只和该单词本身有关。因此，这三个向量都应该由单词的词嵌入得到。另外，每个单词的 query、key、value 不应该是人工指定的，而应该是可学习的。因此，我们可以用可学习的参数来描述从词嵌入到 query、key、value 的变换过程。综上，自注意力的输入 $Q, K, V$ 应该用下面这个公式计算：

$$
\begin{aligned}
Q &= EW^Q \\
K &= EW^K \\
V &= EW^V
\end{aligned}
$$

其中，$E$ 是词嵌入矩阵，也就是每个单词的词嵌入的数组；$W^Q$、$W^K$、$W^V$ 是可学习的参数矩阵。

在 Transformer 中，大部分向量都是用 embedding 来表示的，词嵌入的长度是 $d_{model}$。比如，设输入句子的长度为 $n$，则 $E$ 的形状是 $n \times d_{model}$，$W^Q$、$W^K$ 的形状是 $d_{model} \times d_k$，$W^V$ 的形状是 $d_{model} \times d_v$。

就像卷积层能够用多个卷积核生成多个通道的特征一样，我们也用多组 $W^Q$、$W^K$、$W^V$ 生成多组自注意力结果。这样，每个单词的自注意力表示会更丰富一点。这种机制就叫做**多头注意力**。把多头注意力用在自注意力上的公式为：

$$
\begin{aligned}
\text{head}_i &= \mathrm{Attention}(EW_i^Q, EW_i^K, EW_i^V) \\
\mathrm{MultiHeadSelfAttention}(E)
&= \mathrm{Concat}(\text{head}_1, \ldots, \text{head}_h) W^O
\end{aligned}
$$

> Transformer 论文里默认所有向量都是行向量，参数矩阵都写成了右乘而不是常见的左乘。

其中，$h$ 是多头自注意力的“头数”，$W^O$ 是另一个参数矩阵。多头注意力模块的输入和输出向量的长度都是 $d_{model}$。因此，$W^O$ 的形状是 $hd_v \times d_{model}$（自注意力的输出长度是 $d_v$，有 $h$ 个输出）。

在论文中，Transformer 的默认参数配置如下：

- $d_{model} = 512$
- $h = 8$
- $d_k = d_v = d_{model} / h = 64$

实际上，多头注意力机制不仅仅可以用在计算自注意力上。推广一下，如果把多头注意力的输入 $E$ 拆成三个矩阵 $Q, K, V$，则多头注意力的一般公式为：

$$
\begin{aligned}
\text{head}_i &= \mathrm{Attention}(QW_i^Q, KW_i^K, VW_i^V) \\
\mathrm{MultiHeadAttention}(Q, K, V)
&= \mathrm{Concat}(\text{head}_1, \ldots, \text{head}_h) W^O
\end{aligned}
$$


## Transformer模型架构

看懂了注意力机制，可以学习Transformer的整体架构了。

<img alt="Transformer架构" height="600" src="./images/Transformer/p1.png" width="400"/>

现在仅知道多头注意力模块的原理，对模型主干中的三个模块还有疑问：

1. Add & Norm
2. Feed Forward
3. 为什么一个多头注意力前面加了 掩码（Masked）

### 残差连接

Transformer 使用了和 ResNet 类似的残差连接，即模块本身的映射为 $F(x)$，则模块输出为
$\mathrm{Normalization}(F(x) + x)$。和 ResNet 不同，Transformer 使用的归一化方法是 **LayerNorm**。

另外要注意的是，残差连接有一个要求：输入 $x$ 和输出 $F(x) + x$ 的维度必须等长。在 Transformer 中，包括所有词嵌入在内的向量长度都是 $d_{model} = 512$。


### 前馈网络

架构图中的前馈网络（Feed Forward）其实就是一个全连接网络。具体来说，这个子网络由两个线性层组成，中间用 ReLU 作为激活函数。

$$
\mathrm{FFN}(x) = \max(0, xW_1 + b_1) W_2 + b_2
$$

中间的隐藏层维度记作 $d_{ff}$，默认 $d_{ff} = 2048$。


### 整体架构与掩码多头注意力

论文第 3 章开头介绍了模型的运行原理。和多数强力的序列转换模型一样，Transformer 使用了 **encoder–decoder** 的架构。对于输入序列 $(x_1, \ldots, x_n)$，它会被编码器编码成中间表示
$z = (z_1, \ldots, z_n)$。

在给定 $z$ 的前提下，解码器把输出序列里的单词逐个输出，并把当前得到的输出序列作为输入。也就是说，在第 $t$ 个时刻，解码器的输入是 $(y_1, \ldots, y_t)$，输出下一个单词 $y_{t+1}$（第一个单词 $y_1$ 是句首符 `<SOS>`）。

![解码器输入](./images/Transformer/p6.png)

具体来说，输入序列会经过 $N = 6$ 个结构相同的层。每层由多个子层组成。第一个子层是多头注意力层，准确来说，是**多头自注意力**。这一层可以为每一个输入单词提取出更有意义的表示。之后，数据会经过前馈网络层，最终输出编码结果。

得到 $z$ 之后，要用解码器输出结果。解码器的输入是当前已经生成的序列，该序列会经过一个**掩码（masked）多头自注意力层**。我们暂时不管这个掩码是什么意思，先把它当成普通的多头自注意力层，它的作用和编码器中的一样，用于提取更有意义的表示。

接下来，数据会经过一个**多头注意力层**。这个层比较特别，它的  $K, V$ 来自 $z$，$Q$ 来自上一层的输出。为什么会有这样的设计？这种设计来自早期的注意力模型。在早期的注意力模型中，每一个输出单词都会与每一个输入单词求一个注意力，以找到每一个输出单词最相关的某几个输入单词。用注意力公式来表达的话，$Q$ 就是输出单词，$K, V$ 就是输入单词。

![早期注意力](./images/Transformer/p8.png)

经过第二个多头注意力层后，和编码器一样，数据会经过一个前馈网络。最终，网络输出当前时刻的单词。

乍看之下，为了输出一个序列，我们必须串行调用解码器，逐个生成单词。在测试时，确实如此。但是，在训练时，我们可以直接从数据集里取出正确的输出(y1,y2,...yn)。这样，就不必等上一轮推理结束，可以直接并行地训练了。

这种并行计算有一个要注意的地方。在输出第t+1个单词时，模型不应该提前知道t+1时刻之后的信息。因此，应该只保留时刻之前的信息，遮住后面的输入。这可以通过添加掩码实现。添加掩码的一个示例如下表所示：

| 输入（masked）        | 输出 |
|----------------------|------|
| $(y_1)$              | $y_2$ |
| $(y_1, y_2)$         | $y_3$ |
| $(y_1, y_2, y_3)$    | $y_4$ |


这就是为什么解码器的多头自注意力层前面有一个 **masked**。在论文中，mask 是通过对注意力公式中的 softmax 输入加入 $-\infty$ 来实现的（softmax 的输入为 $-\infty$，注意力权重就几乎为 0，被遮住的输出也几乎为 0）。


### 嵌入层

和其他大多数序列转换任务一样，Transformer主干结构的输入输出都是词嵌入序列。词嵌入，其实就是一个把one-hot向量转换成有意义的向量的转换矩阵。在Transformer中，解码器的嵌入层和输出线性层是共享权重的——输出线性层表示的线性变换是嵌入层的逆变换，其目的是把网络输出的嵌入再转换回one-hot向量。如果某任务的输入和输出是同一种语言，那么编码器的嵌入层和解码器的嵌入层也可以共享权重。

### 位置编码

无论是RNN还是CNN，都能自然地利用到序列的先后顺序这一信息。然而，Transformer的主干网络并不能利用到序列顺序信息。因此，Transformer使用了一种叫做“位置编码”的机制，对编码器和解码器的嵌入输入做了一些修改，以向模型提供序列顺序信息。

嵌入层的输出是一个向量数组，即词嵌入向量的序列。设数组的位置是 $pos$，向量的某一维是 $i$。我们为每一个向量里的每一个数添加一个**位置编码**，这种编码方式需要满足以下性质：

1. 对于同一个 $pos$ 不同的 $i$，即对于一个词嵌入向量的不同元素，它们的编码要各不相同。
2. 对于向量的同一个维度处，不同 $pos$ 的编码不同。且 $pos$ 间要满足相对关系，即
   $$f(pos + 1) - f(pos) = f(pos) - f(pos - 1)。$$

要满足这两种性质的话，我们可以轻松地设计一种编码函数：

$$
\mathrm{Encoding}(pos, i) = \frac{pos}{1000^i}
$$

即对于每一个位置 $i$，用小数点后的 3 个十进制数来表示不同的 $pos$。$pos$ 之间也满足相对关系。

但是，这种编码不利于网络的学习。我们更希望所有编码的数值差距不要太大，且都位于 $0 \sim 1$ 之间。为此，Transformer 使用了**三角函数**作为编码函数。这种位置编码（Positional Encoding，PE）的公式如下：

$$
\begin{aligned}
\mathrm{PE}(pos, 2i) &= \sin\!\left(\frac{pos}{10000^{2i / d_{model}}}\right) \\
\mathrm{PE}(pos, 2i + 1) &= \cos\!\left(\frac{pos}{10000^{2i / d_{model}}}\right)
\end{aligned}
$$

$i$ 不同，则三角函数的周期不同，因此 同$pos$ 不同周期的三角函数值不重复。这满足了上面的性质 1。

另外，根据三角函数的和角公式：

$$
\begin{aligned}
\sin(a + b) &= \sin a \cdot \cos b + \cos a \cdot \sin b \\
\cos(a + b) &= \cos a \cdot \cos b - \sin a \cdot \sin b
\end{aligned}
$$

可以看出，$f(pos + k)$ 是 $f(pos)$ 的一个线性函数，即不同的 $pos$ 之间有相对关系，这满足了性质 2。

本文作者也尝试了用可学习的函数作为位置编码函数。实验表明，二者的表现相当。作者最终选择使用三角函数作为位置编码函数，这是因为三角函数能够外推到任意长度的输入序列，而可学习的位置编码只能适应训练时的序列长度。


## 实验结果

论文同样展示了不同配置下Transformer的消融实验结果。

![实验结果](./images/Transformer/p7.png)

实验 A 表明，在计算量不变的前提下，需要谨慎地调节 $h$ 和 $d_k, d_v$ 的比例，过大或过小都不好。这些实验说明，多头注意力比单头注意力要好。

实验 B 表明，$d_k$ 增加可以提升模型性能。作者认为，这说明计算 key 和 value 的相关性是比较困难的，如果用更精巧的计算方式来代替点乘，可能可以进一步提升性能。

实验 C、D 表明，大模型通常表现更优，并且 dropout 是必要的。

如正文所写，实验 E 探究了**可学习的位置编码**。实验结果显示，可学习的位置编码的效果与三角函数位置编码几乎一致。


## 总结

为了改进RNN不可并行的问题，这篇工作提出了Transformer这一仅由注意力机制构成的模型。Transformer的效果非常出色，不仅训练速度快了，还在两项翻译任务上胜过其他模型。

现在来看，Transformer是近年来最有影响力的深度学习模型之一。它先是在NLP中发扬光大，再逐渐扩散到了CV等领域。文中的一些预测也成为了现实，现在很多论文都在讨论如何在图像中使用注意力，以及如何使用带限制的注意力以降低长序列导致的计算性能问题。